In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query

In [ ]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
from DataPrepare.join_data_preparation import JoinDataPreparator
schema = gen_job_light_imdb_schema('/home/ziniu.wzn/imdb-benchmark')
from DataPrepare.join_data_preparation import JoinDataPreparator
hdf_path = "/home/ziniu.wzn/imdb-benchmark/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

In [ ]:
from Models.pgmpy_BN import Pgmpy_BN, build_meta_info
def train_all(algo="chow-liu", max_parents=1):
    for i, relationship_obj in enumerate(schema.relationships):
        sample_size = 10000000
        relation = relationship_obj.identifier
        df, meta_types, null_values, full_join_est = prep.generate_n_samples(
                        sample_size, relationship_list=[relation], post_sampling_factor=10)
        print(full_join_est)
        print(len(df), len(df.columns))
        print(meta_types)
        meta_info = build_meta_info(df.columns)
        bn = Pgmpy_BN(relation, meta_info, full_join_est)
        model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl"
        print(model_path)
        bn.build_from_data(df, algorithm=algo, max_parents=max_parents, ignore_cols=['id'], sample_size=200000)
        pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        try:
            BN.model = Bn.model.to_junction_tree()
            model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_junction.pkl"
            pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        except:
            print("This BN is not able to transform into junction tree, probably because it's not connected, just use BN")

In [ ]:
train_all("exact", 3)

In [2]:
from Evaluation.parse_query_imdb import prepare_join_queries

In [3]:
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
ensemble_location = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/ensemble_relationships_imdb-light_10000000.pkl"
#query_filename = "/home/ziniu.wzn/imdb-benchmark/cardinality/job_all.sql"
query_filename = "/home/ziniu.wzn/BayesCard/Benchmark/IMDB/job-light.sql"
pairwise_rdc_path = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=False, true_card_exist=True)

In [4]:
parsed_queries[30]

[4050205.0,
 {'bn_index': 4,
  'inverse': False,
  'query': {'title.production_year': (1950.1, 1999.9),
   'movie_companies.movie_companies_nn': 1,
   'movie_companies.company_type_id': 2.0},
  'expectation': ['title.mul_movie_keyword.movie_id']},
 {'bn_index': 3,
  'inverse': False,
  'query': {'title.production_year': (1950.1, 1999.9),
   'movie_keyword.movie_keyword_nn': 1,
   'movie_keyword.keyword_id': 398.0},
  'expectation': []},
 {'bn_index': 3,
  'inverse': True,
  'query': {'title.production_year': (1950.1, 1999.9),
   'movie_keyword.movie_keyword_nn': 1},
  'expectation': []}]

In [ ]:
bn_ensemble = dict()
algo = "clt"
for i in range(5):
    with open(f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl", "rb") as f:
        bn = pickle.load(f)
    bn_ensemble[i] = bn

In [ ]:
bn_ensemble[4].mapping

In [ ]:
sample_size = 10000000
relation = "movie_keyword.movie_id = title.id"
df, meta_types, null_values, full_join_est = prep.generate_n_samples(
                sample_size, relationship_list=[relation], post_sampling_factor=10)
print(full_join_est)
print(len(df), len(df.columns))
print(meta_types)
df.head(10)

In [ ]:
from Models.pgmpy_BN import Pgmpy_BN, build_meta_info
meta_info = build_meta_info(df.columns)
bn = Pgmpy_BN(relation, meta_info, full_join_est)
bn.build_from_data(df, algorithm="chow-liu", max_parents=1, ignore_cols=['id'], sample_size=200000)
model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/3_clt.pkl"
pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
def get_fanout_values(bn, fanout_attrs):
    print(fanout_attrs)
    if len(fanout_attrs) == 1:
        return bn.fanouts[fanout_attrs[0]]
    else:
        fanout_attrs_val = [list(self.fanouts[i]) for i in fanout_attrs]
        return np.asarray(list(itertools.product(*fanout_attrs_val)))

In [ ]:
def expectation(bn, query, fanout_attrs, num_samples=1, coverage=None, return_prob=False, sample_size=10000):
        """
        Calculating the expected value E[P(Q|F)*F]
        Parameters
        ----------
        fanout_attrs: a list of fanout variables F, where we would like to compute the expectation
        Rest parameters: the same as previous function .query().
        """
        if fanout_attrs is None or len(fanout_attrs) == 0:
            return bn.query(query, num_samples, coverage, return_prob, sample_size)
        else:
            query_prob = copy.deepcopy(query)
            probsQ, _ = bn.query(query_prob, num_samples, coverage, True)
            if probsQ == 0:
                if return_prob:
                    return 0, bn.nrows
                else:
                    return 0
            print(f"probsQ {probsQ}")
            
            query, n_distinct = bn.query_decoding(query, coverage)
            if query is None:
                if return_prob:
                    return 0, bn.nrows
                else:
                    return 0
            print(query)
            probsQF = bn.infer_machine.query(fanout_attrs, evidence=query).values
            print(probsQF)
            if any(np.isnan(probsQF)):
                if return_prob:
                    return 0, bn.nrows
                else:
                    return 0
            else:
                probsQF = probsQF / (np.sum(probsQF))
            
            print(np.sum(probsQF*get_fanout_values(bn, fanout_attrs)))
            exp = np.sum(probsQF*get_fanout_values(bn, fanout_attrs)) * probsQ
            if return_prob:
                return exp, bn.nrows
            else:
                return exp*bn.nrows

In [ ]:
import copy
data = df.values
for i in [2,3,4,5,6]:
    if i == 2:
        data_copy = copy.deepcopy(data[:, i])
        data_copy[data_copy==0] = 1
        print(df.columns[i], np.mean(1/data_copy))
    else:
        print(df.columns[i], np.mean(data[:, i]))

In [ ]:
bn.query({"title.kind_id": 7, "title.production_year": [2010, 2011, 2012]})

In [ ]:
bn.fanouts['title.mul_movie_info.movie_id']

In [ ]:
cols = []
for col in df.columns:
    cols.append(col.replace(".", "__"))
df.columns = cols

In [ ]:
a = {"a":1, "b":2}
b = {"b":2, "a":1}

In [ ]:
a == b